In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Union into 3 folders

Since we firstly divided into many folders, we know are merging all folders into 3 categories:
- mandatory logos (mand)
- logos we have chosen (choice)
- the noise, which corrisponded to logos that were not annoted (extra)

In [8]:
base = "/home/labuser/DataModelling/"
l = ["choice", "extra1", "extra2","mand1","mand2","mand3"]
l = ['choice-1']
split = ["/train/", "/test/", "/valid/"]

In [9]:
them_all = [base+r+s for r in l for s in split]
print(them_all)

['/home/labuser/DataModelling/choice-1/train/', '/home/labuser/DataModelling/choice-1/test/', '/home/labuser/DataModelling/choice-1/valid/']


## VISUALIZE DIRECTORY STRUCTURE

Because each model has its own structure, we started off by visualizing the content of a given folder downloaded from Roboflow after its processing.

In [10]:
#here needs to be changed into new directories once they are all united
view = [base + s for s in l if "choice" in s]
view

['/home/labuser/DataModelling/choice-1']

In [11]:
for v in view:
    print(os.listdir(v))

['valid', 'README.dataset.txt', 'test', 'data.yaml', 'README.roboflow.txt', 'train']


In [12]:
os.listdir(view[0]+ "/" + os.listdir(view[0])[0] )

['images', 'labels']

It can be easily inferred that a folder is composed out of the followings:
- unzipped Roboflow folder
    - valid
        - images
        - labels
    - train
        - images
        - labels
    - test
        - images
        - labels
    - READ ME files
    - data.yaml <br>

README can be ignored for our analysis after having read it, since it is just to make the user aware about some info regarding yolo.v5. <br>
We will now analyze what data.yaml is.

In [7]:
for v in view:
    for i in os.listdir(v):
        if "data.yaml" in i:
            path = v + '/' + i
            # Open a file: file
            print(path)
            file = open(path,mode='r')

            # read all lines at once
            all_of_it = file.read()
            print(all_of_it)

            # close the file
            file.close()

/home/labuser/DataModelling/mand1/data.yaml
names:
- Nike
nc: 1
train: LogoDetection-1/train/images
val: LogoDetection-1/valid/images

/home/labuser/DataModelling/mand2/data.yaml
names:
- Adidas
- Nike
nc: 2
train: Logo-Detection-1/train/images
val: Logo-Detection-1/valid/images

/home/labuser/DataModelling/mand3/data.yaml
names:
- Adidas
- Puma
- TheNorthFace
- UnderArmour
- Nike
nc: 5
train: /home/labuser/mand_logos3-1/train/images
val: mand_logos3-1/valid/images



The following file needs to be like this because of stratified sampling:

names:
- Adidas
- Puma
- TheNorthFace
- UnderArmour
- Nike <br>
nc: 5 <br>
train : /home/labuser/data <br>

In [13]:
mand = "/home/labuser/data_yolov5/mand"
#choice = "/home/labuser/data_yolov5/choice_fixed"
extra = "/home/labuser/data_yolov5/extra"

new_directories = [mand, extra]
#new_directories = [choice]

In [1]:
#for directory in new_directories: 
    #os.makedirs(directory)

In [11]:
for directory in new_directories:
    #os.makedirs(directory + "/images")
    #os.makedirs(directory + "/labels")

In [20]:
for path in them_all:
    ## all extras into extra
    if "extra" in path:
        path_im = path + "images/"
        path_l = path + "labels/"
        ds = os.listdir(path_im)
        for d in ds:
            shutil.move(os.path.join(path_im, d), extra + "/images/")
        ds = os.listdir(path_l)
        for d in ds:
            shutil.move(os.path.join(path_l, d), extra + "/labels/")
    #all mands into mand
    elif "mand" in path:
        path_im = path + "images/"
        path_l = path + "labels/"
        ds = os.listdir(path_im)
        for d in ds:
            shutil.move(os.path.join(path_im, d), mand + "/images/")
        ds = os.listdir(path_l)
        for d in ds:
            shutil.move(os.path.join(path_l, d), mand + "/labels/")
            
    #all choice into choice
    elif "choice" in path:
        path_im = path + "images/"
        path_l = path + "labels/"
        ds = os.listdir(path_im)
        for d in ds:
            shutil.move(os.path.join(path_im, d), choice + "/images/")
        ds = os.listdir(path_l)
        for d in ds:
            shutil.move(os.path.join(path_l, d), choice + "/labels/")
        

In [2]:
#os.listdir('/home/labuser/data_yolov5/choice_fixed/images')

# Adding path to the annotation file

We will now proceed by attaching to the annotation file, the path which each image is in, in order to perform the train split later.
TO BE KEPT IN MIND THAT IT IS MISSING IN THE PATH /IMAGES /LABELS since the table only contains images

In [8]:
!pwd

/home/labuser/DataCleaning


In [27]:
annot = pd.read_csv("final_annot.csv")
annot.head()

,photo_filename,width,height,class,xmin,ymin,xmax,ymax
0,charlotte_1019088214340003257_20150630.jpg,640,640,Nike,339,394,381,439
1,charlotte_1056018576661922536_20150820.jpg,640,640,Nike,497,0,574,30
2,charlotte_1057610050380125945_20150823.jpg,1080,1080,Nike,353,406,473,477
3,charlotte_1061715143228072770_20150828.jpg,1080,1349,Nike,13,51,330,198
4,charlotte_1081463278280976819_20150925.jpg,612,612,Nike,485,30,580,69


In [28]:
np.unique(annot["class"])

array(['Adidas', 'AppleInc', 'Chanel', 'CocaCola', 'Emirates',
       'HardRockCafe', 'Intimissimi', 'MercedesBenz', 'NFL', 'Nike',
       'Pepsi', 'Puma', 'RalphLaurenCorporation', 'Starbucks',
       'TheNorthFace', 'Toyota', 'UnderArmour'], dtype=object)

In [29]:
our_logos = ["Adidas","Nike","UnderArmour","Puma","TheNorthFace"]
our_logos = ["AppleInc", "NFL"]
#extra_logos = ["AppleInc","Intimissimi","RalphLaurenCorporation","NFL"]

In [30]:
#directories = ["/home/labuser/data_yolov5/mand", "/home/labuser/data_yolov5/choice", "/home/labuser/data_yolov5/extra"]
directories = ["/home/labuser/data_yolov5/choice"]

In [31]:
pattern_our = "|".join(our_logos)
pattern_extra = "|".join(extra_logos)

In [32]:
mask_our = annot["class"].str.contains(pattern_our)
mask_choice = annot["class"].str.contains(pattern_extra)

annot["path"] = np.nan
#annot["path"][mask_our] = directories[0]
annot["path"][mask_choice] = directories[0]
#annot.fillna(directories[2], inplace = True)

annot.head()

<ipython-input-32-e60e7e4877c1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annot["path"][mask_choice] = directories[0]


,photo_filename,width,height,class,xmin,ymin,xmax,ymax,path
0,charlotte_1019088214340003257_20150630.jpg,640,640,Nike,339,394,381,439,NaN
1,charlotte_1056018576661922536_20150820.jpg,640,640,Nike,497,0,574,30,NaN
2,charlotte_1057610050380125945_20150823.jpg,1080,1080,Nike,353,406,473,477,NaN
3,charlotte_1061715143228072770_20150828.jpg,1080,1349,Nike,13,51,330,198,NaN
4,charlotte_1081463278280976819_20150925.jpg,612,612,Nike,485,30,580,69,NaN


In [33]:
annot[["class","path"]].drop_duplicates().reset_index(drop = True)

,class,path
0,Nike,NaN
1,Adidas,NaN
2,UnderArmour,NaN
3,Puma,NaN
4,TheNorthFace,NaN
5,Starbucks,NaN
6,RalphLaurenCorporation,/home/labuser/data_yolov5/choice
7,AppleInc,/home/labuser/data_yolov5/choice
8,MercedesBenz,NaN
9,NFL,/home/labuser/data_yolov5/choice


# Considering Augumented File

We need to join to the dataframe created, the augumented file as well. We can get the radice out of the augumented and do a join by photo_filename.

Let's start by considering only images, which then will be attached to its text file contained in "./labels" and with the same name but finishing by .txt

In [34]:
them_all = [i + "/images/"  for i in new_directories]
del new_directories
print(them_all)

['/home/labuser/data_yolov5/choice_fixed/images/']


In [35]:
###CREATE A DICTIONARY CONTAINING THE NAMES OF THE AUGUMENTED FILES
content_list = {}
for index, path in enumerate(them_all):
    #os.path.join(base, val)
    content_list[ them_all[index] ] = os.listdir(path)



In [36]:
content_list

{'/home/labuser/data_yolov5/choice_fixed/images/': ['miami_1927361813471030367_20181205_jpg.rf.665bc8d07ecbc40d976806693736daa9.jpg',
  'london_1841477276598031696_20180808_jpg.rf.0e49f7b09241b520202cb16afebd23f7.jpg',
  'charlotte_1837248889205815320_20180802_jpg.rf.41777068cc74566c315d5719451076dd.jpg',
  'charlotte_1621196505167522464_20171008_jpg.rf.37f01374ec2f3a294663eaa06db766ae.jpg',
  'charlotte_1837225445276616653_20180802_jpg.rf.d67763891fa158b8cf84e6908d438b99.jpg',
  'cleveland_1616265762878946521_20171001_jpg.rf.b243f63c968672c780bf0e60820608e8.jpg',
  'charlotte_1169979447456882783_20160125_jpg.rf.77bf50142cb4dd40c64cb86f5bc6c99c.jpg',
  'charlotte_1169979447456882783_20160125_jpg.rf.f495966bd7b5be256286443bf0ce9dc4.jpg',
  'la_1195255318239608642_20160229_jpg.rf.90011a291e7ae9920cd1c81d45082ef5.jpg',
  'houston_1721175268977814375_20180223_jpg.rf.99721bb3a1a45c83ea40fa15d646462c.jpg',
  'houston_1578427553851676696_20170810_jpg.rf.6a3f36722bf42b47d811770a28781494.jpg',


In [37]:
## modify the created dictionary into a dataframe

df = pd.DataFrame.from_dict(content_list, orient = 'index').T
df_tot = pd.DataFrame(columns = ["path", "images"])
for index in df.columns:
    images = df.loc[:,index].dropna()
    p = [str(index)]
    path = p * len(images)
    trash = pd.DataFrame([np.array(path).T, images]).T
    trash.columns = ["path", "images"]
    df_tot = pd.concat([df_tot, trash])
df_tot.reset_index(inplace = True, drop = True)
del df
del trash

In [38]:
np.unique(df_tot["path"])

array(['/home/labuser/data_yolov5/choice_fixed/images/'], dtype=object)

## Needed a join with annot

In [39]:
df_tot.shape, annot.shape

((5814, 2), (46163, 9))

In [40]:
df_tot["photo_filename"] = [col[0] for col in df_tot["images"].str.split("_jpg.rf.")]

In [41]:
### removal of .jpg and images/ to add easily after
df_tot["images"] = df_tot['images'].str.slice(stop = -4)
df_tot["path"] = df_tot['path'].str.slice(stop = -7)

In [42]:
annot["photo_filename"] = annot["photo_filename"].str.slice(stop = -4)

In [43]:
df_tot.head()

,path,images,photo_filename
0,/home/labuser/data_yolov5/choice_fixed/,miami_1927361813471030367_20181205_jpg.rf.665b...,miami_1927361813471030367_20181205
1,/home/labuser/data_yolov5/choice_fixed/,london_1841477276598031696_20180808_jpg.rf.0e4...,london_1841477276598031696_20180808
2,/home/labuser/data_yolov5/choice_fixed/,charlotte_1837248889205815320_20180802_jpg.rf....,charlotte_1837248889205815320_20180802
3,/home/labuser/data_yolov5/choice_fixed/,charlotte_1621196505167522464_20171008_jpg.rf....,charlotte_1621196505167522464_20171008
4,/home/labuser/data_yolov5/choice_fixed/,charlotte_1837225445276616653_20180802_jpg.rf....,charlotte_1837225445276616653_20180802


In [44]:
annot.head()

,photo_filename,width,height,class,xmin,ymin,xmax,ymax,path
0,charlotte_1019088214340003257_20150630,640,640,Nike,339,394,381,439,NaN
1,charlotte_1056018576661922536_20150820,640,640,Nike,497,0,574,30,NaN
2,charlotte_1057610050380125945_20150823,1080,1080,Nike,353,406,473,477,NaN
3,charlotte_1061715143228072770_20150828,1080,1349,Nike,13,51,330,198,NaN
4,charlotte_1081463278280976819_20150925,612,612,Nike,485,30,580,69,NaN


In [45]:
df = df_tot.merge(annot[["class","photo_filename"]], on = ["photo_filename"], how = "left")
del df_tot

In [46]:
df.head()

,path,images,photo_filename,class
0,/home/labuser/data_yolov5/choice_fixed/,miami_1927361813471030367_20181205_jpg.rf.665b...,miami_1927361813471030367_20181205,Intimissimi
1,/home/labuser/data_yolov5/choice_fixed/,london_1841477276598031696_20180808_jpg.rf.0e4...,london_1841477276598031696_20180808,Intimissimi
2,/home/labuser/data_yolov5/choice_fixed/,charlotte_1837248889205815320_20180802_jpg.rf....,charlotte_1837248889205815320_20180802,RalphLaurenCorporation
3,/home/labuser/data_yolov5/choice_fixed/,charlotte_1621196505167522464_20171008_jpg.rf....,charlotte_1621196505167522464_20171008,NFL
4,/home/labuser/data_yolov5/choice_fixed/,charlotte_1837225445276616653_20180802_jpg.rf....,charlotte_1837225445276616653_20180802,RalphLaurenCorporation


## DIVISION INTO TRAIN AND TEST

In [47]:
our_logos

['AppleInc', 'NFL']

In [48]:
extra_logos

['AppleInc', 'Intimissimi', 'RalphLaurenCorporation', 'NFL']

In [49]:
logos_tot = np.unique(annot["class"])
print(logos_tot)

['Adidas' 'AppleInc' 'Chanel' 'CocaCola' 'Emirates' 'HardRockCafe'
 'Intimissimi' 'MercedesBenz' 'NFL' 'Nike' 'Pepsi' 'Puma'
 'RalphLaurenCorporation' 'Starbucks' 'TheNorthFace' 'Toyota'
 'UnderArmour']


In [50]:
extra_logos = []

In [52]:
target_dir_train = "/home/labuser/data_yolov5/train"
######????????? trainextra va messsa????
target_dir_trainextra = "/home/labuser/data_yolov5/trainextra"
target_dir_test = "/home/labuser/data_yolov5/test"
target_dir_testextra = "/home/labuser/data_yolov5/testextra"
target_dir_val = "/home/labuser/data_yolov5/val"
target_dir_valextra = "/home/labuser/data_yolov5/valextra"

both = ["/images/", "/labels/"]

new_directories_ = [target_dir_train,
                    ########????????
                    target_dir_trainextra,
                    target_dir_test, 
                   target_dir_testextra, target_dir_val,
                   target_dir_valextra]

new_directories = [i + k for i in new_directories_ for k in both]


#for directory in new_directories: 
#    os.makedirs(directory)

for logo in logos_tot: 
    df_temp = df[df["class"] == logo]
            
            
    if logo in our_logos: 
        ## TODO: test should really be added or data scraping?
        train, test_ = train_test_split(df_temp, test_size = 0.3)
        val, test = train_test_split(test_, test_size = 0.33)
        
        
        ## move the images belonging to train to train folder
        for i in range(train.shape[0]):

            src = train.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = train.iloc[i]['images'] 
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_train + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_train + both[1])
        
        ## move the images belonging to test to test folder
        
        for i in range(test.shape[0]):

            src = test.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = test.iloc[i]['images'] 
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_test + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_test + both[1])
        
              
              
        ## move the images belonging to val to val folder
        
        
        for i in range(val.shape[0]):

            src = val.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = val.iloc[i]['images'] 
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_val + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_val + both[1])
    
   
        
    elif logo in extra_logos: 
        train, test_ = train_test_split(df_temp, test_size = 0.3)
        val, test = train_test_split(test_, test_size = 0.33)


        ## move the images belonging to train to train folder
        for i in range(train.shape[0]):

            src = train.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = train.iloc[i]['images'] 
            #####????????????????? trainextra
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_trainextra + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_trainextra + both[1])
            ##### ALTERNATIVE
           # shutil.move(os.path.join(src, file_name + 'jpg'), target_dir_train)
           # shutil.move(os.path.join(src, file_name + 'txt'), target_dir_train)

        ## move the images belonging to test to test folder

        for i in range(test.shape[0]):

            src = test.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = test.iloc[i]['images'] 
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_testextra + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_testextra + both[1])


        ## move the images belonging to val to val folder


        for i in range(val.shape[0]):

            src = val.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = val.iloc[i]['images'] 
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_valextra + both[0])
            shutil.move(os.path.join(src_l , file_name + '.txt'), target_dir_valextra + both[1])




    else: 
        for i in range(df_temp.shape[0]):

            src = df_temp.iloc[i]['path']
            src_im = src + "images/"
            src_l = src + "labels/"
            file_name = df_temp.iloc[i]['images']
            shutil.move(os.path.join(src_im, file_name + '.jpg'), target_dir_train + both[0])
            shutil.move(os.path.join(src_l, file_name + '.txt'), target_dir_train + both[1])